In [95]:
import pandas as pd

main_df = pd.read_csv(r"data\ner_data_20_5_part3.csv")
main_df

,Sentence,Label
0,HUT LG CLASSIC NO GR~~HUT LARGE PIZZA~~L SF NG...,"['O', 'S', 'B', 'O', 'O', 'O', 'S', 'C', 'O', ..."
1,PREP BASE SAUCE/CHS~~PREP BUILD~~L SC 170G PLA...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,UE. DEALS~~UE. DEALS~~UE C2 DEAL 1 MEDIUM PIZZA,"['O', 'C', 'O', 'C', 'O', 'O', 'O', 'O', 'S', ..."
3,DV. LG CLASSIC NO GR~~DV. LARGE PIZZA~~LARGE S...,"['O', 'S', 'B', 'O', 'O', 'O', 'S', 'C', 'S', ..."
4,UE. PREMIUM SIDES~~UE. SIDES~~HOT HONEY BBQ CH...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...
95,JE. TOPPINGS DEFAULT~~JE. TOPPINGS~~P PLANT (E...,"['O', 'C', 'O', 'O', 'C', 'O', 'O', 'O', 'O', ..."
96,DV. MED PIZZA~~DV. MED PIZZA~~MEDIUM DEAL 1 PL...,"['O', 'S', 'C', 'O', 'S', 'C', 'S', 'C', 'O', ..."
97,0007-SOFT BEVERAGE~~PURCHASED ITEMS~~JIMMY'S I...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"
98,UE. LG PAN~~UE. LARGE PIZZA~~YUM L PAN MEAT SU...,"['O', 'S', 'B', 'O', 'S', 'C', 'O', 'O', 'B', ..."


In [43]:
# validating tags length and sentence length in dataset given by gpt-4

from ast import literal_eval

def validating_data(dataframe: pd.DataFrame) -> dict[str, int | list[int]]:
    unequal_rows = []
    for idx in range(len(dataframe)):
        length_words = len(" ".join(tuple(dataframe.iloc[idx,:].values)[0].split("~~")).strip().split(" "))
        length_tags = len(literal_eval(tuple(dataframe.iloc[idx,:].values)[1]))

        if length_words != length_tags:
            unequal_rows.append(idx)

    return {"rows": unequal_rows, "length": len(unequal_rows)}


In [44]:
# Combining all small datasets together to make one dataset
import os
import pandas as pd

directory = "data"
dataframe = pd.DataFrame()

files = os.listdir(directory)

for element in files:
    df = pd.read_csv(os.path.join(directory, element))
    dataframe = pd.concat([dataframe, df], ignore_index=True)


dataframe

,Sentence,Label
0,HUT INSTRUCTION~~HUT DEALS~~,"['O', 'O', 'O', 'C']"
1,HUT SML CLASSIC~~HUT SMALL PIZZA~~,"['O', 'S', 'B', 'O', 'S', 'C']"
2,0001-PIZZA FOOD~~PURCHASED ITEMS~~RAPESEED OIL...,"['C', 'O', 'O', 'O', 'O', 'O', 'O']"
3,HUT INSTRUCTION~~HUT DEALS~~,"['O', 'O', 'O', 'C']"
4,HUT INSTRUCTION~~HUT DEALS~~,"['O', 'O', 'O', 'C']"
...,...,...
275,JE. MED PIZZA~~JE. MED PIZZA~~MEDIUM DEAL 1 PL...,"['O', 'S', 'C', 'O', 'S', 'C', 'S', 'O', 'O', ..."
276,UE. MED CLASSIC NO G~~UE. MED PIZZA~~M CLC NG ...,"['O', 'S', 'B', 'O', 'O', 'O', 'S', 'C', 'O', ..."
277,PHD SML CLASSIC NO G~~PHD SMALL PIZZA~~SMALL S...,"['O', 'S', 'B', 'O', 'O', 'O', 'S', 'C', 'S', ..."
278,HUT TOPPINGS DEFAULT~~HUT TOPPINGS~~S CLC 1G H...,"['O', 'C', 'O', 'O', 'C', 'O', 'O', 'O', 'O', ..."


#### Converting labels to ner_tags (str2int)

In [110]:
import pandas as pd
from datasets import Dataset
from ast import literal_eval

sentences = [{"text": dataframe["Sentence"][i] , "labels": literal_eval(dataframe["Label"][i])}  for i in range(len(dataframe))]

ds = Dataset.from_pandas(pd.DataFrame(data=sentences))
ds["labels"][0]



['O', 'O', 'O', 'C']

In [111]:
from datasets import ClassLabel

# Define a Classlabel object to use to map string labels to integers.
classmap = ClassLabel(num_classes=4, names=['O', 'C', 'S', 'B'])

# Map labels to label ids.
ds = ds.map(lambda y: {"labels": classmap.str2int(y["labels"])})

Map: 100%|██████████| 280/280 [00:00<00:00, 11666.12 examples/s]


In [113]:
ds["labels"]

[[0, 0, 0, 1],
 [0, 2, 3, 0, 2, 1],
 [1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 2, 3, 0, 2, 1],
 [0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 0],
 [0, 2, 3, 0, 2, 1],
 [0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 2, 3, 0, 2, 1],
 [0, 0, 0, 1],
 [0, 2, 3, 0, 2, 1],
 [0, 0, 0, 1],
 [0, 2, 3, 0, 2, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 2, 3, 0, 2, 1],
 [0, 2, 1, 0, 2, 1, 2, 1, 0, 0, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 2, 1, 0, 0, 0, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 2, 0, 1, 0, 0, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 2, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 2, 0, 0, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 2, 1, 0, 0, 0, 0, 0, 0],
 [0, 2, 3, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 3, 0],
 [0, 2, 1, 0, 2, 1, 2, 0, 0, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 2, 1, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 2, 0, 1, 0, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 2, 1, 0, 0, 0, 0],
 [0, 2, 1, 0, 2, 1, 2, 0, 0, 0]

In [45]:
result = validating_data(dataframe)
result["rows"]

[108, 110, 149]

In [46]:
clean_df = dataframe.drop(result["rows"], axis=0).reset_index()

In [47]:
clean_df.drop("index", axis=1, inplace=True)

In [55]:
# import pandas as pd
# from datasets import Dataset
# from ast import literal_eval

# sentences = [{"text": df["Sentence"][i] , "labels": literal_eval(df["Label"][i])}  for i in range(len(df))]

# ds = Dataset.from_pandas(pd.DataFrame(data=sentences))


In [56]:
# from datasets import ClassLabel
# from transformers import AutoTokenizer

# # Define a tokenizer
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# # Define a Classlabel object to use to map string labels to integers.
# classmap = ClassLabel(num_classes=4, names=['O', 'C', 'S', 'B'])

# # Map text to tokenizer ids.
# ds = ds.map(lambda x: tokenizer(x["text"], truncation=True))

# # Map labels to label ids.
# ds = ds.map(lambda y: {"labels": classmap.str2int(y["labels"])})


Map: 100%|██████████| 150/150 [00:00<00:00, 11539.30 examples/s]


In [48]:
len(clean_df)

277

In [49]:
import math
training_dataset_size = math.ceil(len(clean_df) * 0.8)
print(training_dataset_size)

222


In [60]:
import pandas as pd
import evaluate
from ast import literal_eval
import math

from datasets import Dataset
from datasets import ClassLabel

from transformers import AutoModelForTokenClassification, Trainer, AutoTokenizer, DataCollatorForTokenClassification

model_name = "bert-base-uncased"

# Define a Classlabel object to use to map string labels to integers.
classmap = ClassLabel(num_classes=4, names=['O', 'C', 'S', 'B'])

total_sentences = [{"text": clean_df["Sentence"][i] , "labels": literal_eval(clean_df["Label"][i])}  for i in range(len(clean_df))]

training_dataset_size = math.ceil(len(clean_df) * 0.8)

train_sentences = total_sentences[:training_dataset_size]

eval_sentences = total_sentences[training_dataset_size + 1:]

ds_train = Dataset.from_pandas(pd.DataFrame(data=train_sentences))
ds_eval = Dataset.from_pandas(pd.DataFrame(data=eval_sentences))

model = AutoModelForTokenClassification.from_pretrained(model_name,
                                                        id2label={i:classmap.int2str(i) for i in range(classmap.num_classes)},
                                                        label2id={c:classmap.str2int(c) for c in classmap.names},
                                                        finetuning_task="ner")
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorForTokenClassification(tokenizer)


ds_train = ds_train.map(lambda x: tokenizer(x["text"], truncation=True))
ds_eval = ds_eval.map(lambda x: tokenizer(x["text"], truncation=True))

ds_train = ds_train.map(lambda y: {"labels": classmap.str2int(y["labels"])})
ds_eval = ds_eval.map(lambda y: {"labels": classmap.str2int(y["labels"])})


metric = evaluate.load("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Initialize our Trainer
trainer = Trainer(
    model=model,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

Map: 100%|██████████| 222/222 [00:00<00:00, 8222.53 examples/s]

Map: 100%|██████████| 54/54 [00:00<00:00, 6738.44 examples/s]

Map: 100%|██████████| 222/222 [00:00<00:00, 16290.84 examples/s]

Map: 100%|██████████| 54/54 [00:00<00:00, 10645.94 examples/s]
d:\ner model advanced\env\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batc

{'train_runtime': 198.4413, 'train_samples_per_second': 3.356, 'train_steps_per_second': 0.423, 'train_loss': 0.4551026480538504, 'epoch': 3.0}


TrainOutput(global_step=84, training_loss=0.4551026480538504, metrics={'train_runtime': 198.4413, 'train_samples_per_second': 3.356, 'train_steps_per_second': 0.423, 'train_loss': 0.4551026480538504, 'epoch': 3.0})

In [88]:
ds_train

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 222
})

In [51]:
clean_df.to_csv("ner_data_complete.csv", index = False)

In [80]:
tokenizer.tokenize('UE. LG PIZZA~~UE. LARGE PIZZA~~L SF MEAT SUPREME CRUST')

['u',
 '##e',
 '.',
 'l',
 '##g',
 'pizza',
 '~',
 '~',
 'u',
 '##e',
 '.',
 'large',
 'pizza',
 '~',
 '~',
 'l',
 'sf',
 'meat',
 'supreme',
 'crust']

In [96]:
main_df["Sentence"][0]

'HUT LG CLASSIC NO GR~~HUT LARGE PIZZA~~L SF NG HOT HONEY PEPPERONI FEAST'

In [124]:
inputs = tokenizer(main_df["Sentence"][0])
inputs.tokens()

['[CLS]',
 'hut',
 'l',
 '##g',
 'classic',
 'no',
 'gr',
 '~',
 '~',
 'hut',
 'large',
 'pizza',
 '~',
 '~',
 'l',
 'sf',
 'ng',
 'hot',
 'honey',
 'pepper',
 '##oni',
 'feast',
 '[SEP]']

In [125]:
inputs.word_ids()

[None,
 0,
 1,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 17,
 18,
 None]

In [164]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        # print(word_id)
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            # print("loop word id", word_id)
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
            # print(new_labels)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            # if label % 2 == 1:
            #     print("else")
            #     label += 1
            new_labels.append(label)

    return new_labels


In [175]:
tokenizer(" ".join(ds["text"][101].split("~~")).strip()).tokens()

['[CLS]',
 '000',
 '##2',
 '-',
 'pizza',
 'topping',
 '##s',
 'purchased',
 'items',
 'z',
 '##tur',
 '##key',
 'stamp',
 '[SEP]']

In [170]:
labels = ds["labels"][0]
sentence = " ".join(ds["text"][0].split("~~")).strip()
inputs = tokenizer(sentence)
word_ids = inputs.word_ids()
tokens = inputs.tokens()
print(word_ids)
print(tokens)
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[None, 0, 1, 2, 3, None]
['[CLS]', 'hut', 'instruction', 'hut', 'deals', '[SEP]']
[0, 0, 0, 1]
[-100, 0, 0, 0, 1, -100]


In [79]:
len(inputs["input_ids"])

22

In [73]:
len(list(ds_eval)[0]["input_ids"])

22

In [53]:
predictions = trainer.predict(ds_eval)

NameError: name 'label_list' is not defined

In [65]:
model.save_pretrained("finetuned_ner")

In [66]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

new_model = AutoModelForTokenClassification.from_pretrained("finetuned_ner")
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [67]:
n = 90

input_text = clean_df["Sentence"][n]

print("Text:", input_text)

print("Actual Labels:", clean_df["Label"][n])
input_text_preprocessed = " ".join(input_text.split("~~")).strip()
print(input_text_preprocessed)

# Prepare input
inputs = tokenizer(input_text_preprocessed, return_tensors="pt", truncation=True)

# Perform inference
outputs = new_model(**inputs)

# Post-process output
predicted_labels = outputs.logits.argmax(dim=2)

def indices_to_labels(indices, classmap):
    labels = [classmap.int2str(idx) for idx in indices]
    return labels

predicted_labels_classes = indices_to_labels(predicted_labels.squeeze().tolist(), classmap)

print("Predicted Labels:", predicted_labels_classes)

Text: UE. LG PIZZA~~UE. LARGE PIZZA~~LARGE MEAT SUPREME
Actual Labels: ['O', 'S', 'C', 'O', 'S', 'C', 'S', 'O', 'O']
UE. LG PIZZA UE. LARGE PIZZA LARGE MEAT SUPREME
Predicted Labels: ['O', 'S', 'C', 'O', 'S', 'C', 'S', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B']


In [82]:
print(tokenizer.tokenize("UE. LG PIZZA~~UE. LARGE PIZZA~~LARGE MEAT SUPREME"))

['u', '##e', '.', 'l', '##g', 'pizza', '~', '~', 'u', '##e', '.', 'large', 'pizza', '~', '~', 'large', 'meat', 'supreme']


In [68]:
len(predicted_labels_classes)

22